In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName('test') \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/24 09:32:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/24 09:32:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-02.parquet

--2022-11-24 09:33:11--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:21d6:1000:b:20a5:b140:21, 2600:9000:21d6:800:b:20a5:b140:21, 2600:9000:21d6:d800:b:20a5:b140:21, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:21d6:1000:b:20a5:b140:21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 302633211 (289M) [application/x-www-form-urlencoded]
Saving to: ‘fhvhv_tripdata_2021-02.parquet’

fhvhv_tripdata_2021 100%[===================>] 288,61M  45,9MB/s    in 6,7s    

2022-11-24 09:33:19 (43,2 MB/s) - ‘fhvhv_tripdata_2021-02.parquet’ saved [302633211/302633211]



In [16]:
df = spark.read \
    .option("header", "true") \
    .parquet('fhvhv_tripdata_2021-02.parquet')

In [5]:
df = df.repartition(24)

In [6]:
df.write.parquet('fhvhv/2021/02/')

22/11/24 09:37:27 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


22/11/24 09:37:38 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


22/11/24 09:37:38 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


22/11/24 09:37:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


22/11/24 09:37:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [14]:
# What's the size of the folder with results (in MB)?
import os

# assign size
size = 0
 
# assign folder path
Folderpath = 'fhvhv/2021/02/'
 
# get size
for ele in os.scandir(Folderpath):
    size+=os.path.getsize(ele)
     
print(f"Size: {size/(1024*1024)} MB")

print(f"Original size: {os.stat('fhvhv_tripdata_2021-02.parquet').st_size/(1024*1024)}")

# https://stackoverflow.com/questions/54218006/why-does-the-repartition-method-increase-file-size-on-disk

Size: 515.6801538467407 MB
Original size: 288.6135206222534


In [18]:
df = spark.read.parquet('fhvhv/2021/02/')

In [19]:
from pyspark.sql import functions as F

In [24]:
# How many taxi trips were there on February 15?
df.filter(F.to_date(df.pickup_datetime) == '2021-02-15').count()

367170